In [90]:
!pip install pandas

In [91]:
import pandas as pd
my_file_path = r"C:\Users\user\Desktop\iit\year2 sem1\sdgp\AgroEdge\farming data2.csv"
farming_data=pd.read_csv(my_file_path)
farming_data.head()

,Farmer Name,Farm ID,Farm Size (Perches),Farm Type,Farm Location (Village),Farm Location (District),Crop Type,Brinjal Variety,Planting Start Date,Planting End Date,...,Total Water Usage (liters),Irrigation Schedule,Irrigation Cost (Rs.),Total Number of Labor Hours,Total Labor Wages,Market Selling Price (Rs./kg),Total Sales Revenue (Rs.),Profit/Loss (Rs.),Buyer Type,Storage Method
0,Eran Wickramasinghe,6710,9.0,Organic,Hanwella,Colombo,Brinjal,Local,27/10/2024,20/01/2025,...,6583.16,Daily,4045.21,8,2453.45,154.11,40680.42,4040.16,Retail,Refrigeration
1,Chandana Fernando,7527,112.0,Mixed,Minuwangoda,Gampaha,Brinjal,Other,07/05/2024,28/08/2024,...,936.96,Weekly,1735.96,8,1399.81,100.71,27617.70,8243.59,Retail,Refrigeration
2,Amara Perera,4828,182.4,Conventional,Bulathsinhala,Kalutara,Brinjal,Hybrid,27/07/2024,16/11/2024,...,1991.83,Daily,3469.49,9,2561.95,371.97,109366.62,76595.58,Direct Consumers,Refrigeration
3,Chandana Fernando,5629,352.0,Mixed,Pussellawa,Kandy,Brinjal,Hybrid,23/06/2023,04/10/2023,...,2162.38,Based on moisture levels,643.28,5,2609.61,444.54,87294.32,67207.80,Retail,Refrigeration
4,Dinesh Kumara,5543,441.6,Mixed,Rattota,Matale,Brinjal,Other,14/03/2023,21/05/2023,...,5577.55,Based on moisture levels,3593.12,8,2025.32,313.02,139995.06,111829.22,Wholesale,Refrigeration


In [92]:
print(farming_data.isnull().sum())

Farmer Name                         0
Farm ID                             0
Farm Size (Perches)                 0
Farm Type                           0
Farm Location (Village)             0
Farm Location (District)            0
Crop Type                           0
Brinjal Variety                     0
Planting Start Date                 0
Planting End Date                   0
Expected Harvest (kg)               0
Actual Harvest (kg)                 0
Soil Type                           0
Soil pH Level                       1
Seed Type                           0
Seed Variety                        0
Seed Source                         0
Seed Quantity (kg)                  0
Seed Cost (Rs.)                     0
Fertilizer Type                     0
Fertilizer Brand                    0
Total Fertilizer Quantity (kg)      0
Fertilizer N-P-K Ratio              0
Fertilizer Application Schedule     0
Fertilizer Application Method       0
Fertilizer Cost (Rs.)               0
Pesticide Ty

In [93]:
import numpy as np
from scipy.stats import zscore

logical_min_threshold = 10 #asume the smallest value that can be farmsize

# Compute IQR (Interquartile Range)
Q1 = farming_data['Farm Size (Perches)'].quantile(0.25)  # 25th percentile
Q3 = farming_data['Farm Size (Perches)'].quantile(0.75)  # 75th percentile

IQR = Q3 - Q1   #calculate IQR

upper_bound_iqr= Q3 + 1.2 * IQR # Compute an upper bound dynamically

#calculate zcore
z_scores = zscore(farming_data['Farm Size (Perches)']) 

# Identify outliers based on above criteria
outlier_indices = farming_data.index[
    (farming_data['Farm Size (Perches)'] < logical_min_threshold) |  # Very small values
    (farming_data['Farm Size (Perches)'] > upper_bound_iqr) |        # IQR-based high outliers
    (z_scores > 2.5)                                                 # Z-score-based extreme values
]
removed_outliers = farming_data.loc[outlier_indices, ['Farm Size (Perches)']]
#remove those outliers 
farming_data.loc[outlier_indices, 'Farm Size (Perches)'] = np.nan
print(removed_outliers)


    Farm Size (Perches)
0                   9.0
11                  8.0
12                  7.0
49                  8.0
59                  7.0
75                  6.0
85                  9.0


In [94]:
import pandas as pd
import numpy as np
from scipy.stats import zscore

# Avoid division errors (if Farm Size is 0 or NaN)
valid_farm_data = farming_data[farming_data['Farm Size (Perches)'] > 0]

# Compute Expected Yield per Perch dynamically (without adding a new column)
expected_yield_per_perch = valid_farm_data['Expected Harvest (kg)'] / valid_farm_data['Farm Size (Perches)']

# Compute IQR for 'Expected Yield per Perch'
Q1 = expected_yield_per_perch.quantile(0.25)
Q3 = expected_yield_per_perch.quantile(0.75)
IQR = Q3 - Q1

# Compute IQR-based limits
upper_bound_iqr = Q3 + 1.5 * IQR  # Upper threshold


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [ ]:
import pandas as pd
# Define the updated function with rounding to two decimal places
def fill_soil_ph(row):
    if pd.isnull(row['Soil pH Level']):  
        # Filter data for the same village
        village_data = farming_data[farming_data['Farm Location (Village)'] == row['Farm Location (Village)']]
        
        # Check if the village has at least 2 records with valid data
        if village_data['Soil pH Level'].count() >= 2:
            village_mean = round(village_data['Soil pH Level'].mean(skipna=True), 2)
            print(f"Filling missing value using village mean ({row['Farm Location (Village)']}): {village_mean}")
            return village_mean
        
        # If no valid data in the village, check the district
        district_data = farming_data[farming_data['Farm Location (District)'] == row['Farm Location (District)']]
        
        # Check if the district has at least 2 records with valid data
        if district_data['Soil pH Level'].count() >= 2:
            district_mean = round(district_data['Soil pH Level'].mean(skipna=True), 2)
            print(f"Filling missing value using district mean ({row['Farm Location (District)']}): {district_mean}")
            return district_mean
        
        # If neither village nor district has at least 2 valid records, return the global mean
        global_mean = round(farming_data['Soil pH Level'].mean(skipna=True), 2)
        print(f"Filling missing value using global mean: {global_mean}")
        return global_mean
    
    return row['Soil pH Level']  

# Apply function to fill missing values
farming_data['Soil pH Level'] = farming_data.apply(fill_soil_ph, axis=1)

# Check if any NaN values remain
missing_values = farming_data['Soil pH Level'].isnull().sum()

# Print the total number of missing values left
print(f"Total missing values remaining: {missing_values}")

In [ ]:
print(farming_data.isnull().sum())

In [ ]:
df_kandy = farming_data[farming_data["Farm Location (District)"] == "Kandy"]

# Get the count and sum of pH values
ph_count = df_kandy["Soil pH Level"].count()
ph_sum = df_kandy["Soil pH Level"].sum()
print(ph_count)
print(ph_sum)

In [ ]:
print("Duplicate rows:",farming_data.duplicated().sum())

In [ ]:
farming_data= farming_data.drop_duplicates()

In [ ]:
print("Duplicate rows:",farming_data.duplicated().sum())


In [ ]:
farming_data.to_csv("cleaned_farmer_data.csv", index=False)


In [ ]:
#calculation for to give score by each creteria trough cleaned_farmer_data.csv
clean_farming_data= pd.read_csv("cleaned_farmer_data.csv")

In [ ]:
import numpy as np
# Calculation of Yield per Acre
clean_farming_data['Yield per Acre'] = clean_farming_data['Actual Harvest (kg)'] / clean_farming_data['Farm Size (Perches)']

# Calculation for Cost per kg of production
clean_farming_data['Cost per kg'] = clean_farming_data['Total Labor Wages'] / clean_farming_data['Actual Harvest (kg)']

# Calculation for Soil and Seed Performance
k = 1.5  # Scaling factor
clean_farming_data['Soil Suitability Score'] = 100 * np.exp(-k * abs(clean_farming_data['Soil pH Level'] - 6.2))

# Calculation for Seed Efficiency
clean_farming_data['Seed Efficiency'] = clean_farming_data['Actual Harvest (kg)'] / clean_farming_data['Seed Quantity (kg)']

# Calculation for Fertilizer Efficiency
clean_farming_data['Fertilizer Efficiency'] = clean_farming_data['Actual Harvest (kg)'] / clean_farming_data['Total Fertilizer Quantity (kg)']

# Calculation for Fertilizer Cost per kg of Yield
clean_farming_data['Fertilizer Cost per kg'] = clean_farming_data['Fertilizer Cost (Rs.)'] / clean_farming_data['Actual Harvest (kg)']

# Calculation for Pesticide Efficiency
clean_farming_data['Pesticide Efficiency'] = clean_farming_data['Actual Harvest (kg)'] / clean_farming_data['Total Pesticide Quantity (kg)']

# Calculation for Pesticide Cost per kg of Yield
clean_farming_data['Pesticide Cost per kg'] = clean_farming_data['Pesticide Cost (Rs.)'] / clean_farming_data['Actual Harvest (kg)']

# Calculation for Water Efficiency
clean_farming_data['Water Efficiency'] = clean_farming_data['Actual Harvest (kg)'] / clean_farming_data['Total Water Usage (liters)']

# Calculation for Water Cost per kg of Yield
clean_farming_data['Water Cost per kg'] = clean_farming_data['Irrigation Cost (Rs.)'] / clean_farming_data['Actual Harvest (kg)']

# Calculation for Labor Efficiency
clean_farming_data['Labor Efficiency'] = clean_farming_data['Actual Harvest (kg)'] / clean_farming_data['Total Number of Labor Hours']
#calculate mean of the market price
market_avg_price = clean_farming_data['Market Selling Price (Rs./kg)'].mean()
# Calculation for Market Price Score
clean_farming_data['Market Price Score'] = (clean_farming_data['Market Selling Price (Rs./kg)'] / market_avg_price) * 100
#Calculate Total Sales Revenue per acre
clean_farming_data['Total Sales Revenue per acre (Rs.)'] = clean_farming_data['Total Sales Revenue (Rs.)'] / clean_farming_data['Farm Size (Perches)']
# Calculate Best Revenue per acre
best_revenue_per_acre = clean_farming_data['Total Sales Revenue per acre (Rs.)'].max()
# Calculation for Revenue Score
clean_farming_data['Revenue Score per acre'] = (clean_farming_data['Total Sales Revenue per acre (Rs.)'] / best_revenue_per_acre) * 100

#calculate ROI
clean_farming_data['Total Investment (Rs.)'] = (
    clean_farming_data['Seed Cost (Rs.)'] +
    clean_farming_data['Fertilizer Cost (Rs.)'] +
    clean_farming_data['Pesticide Cost (Rs.)'] +
    clean_farming_data['Irrigation Cost (Rs.)'] +
    clean_farming_data['Total Labor Wages']
)
clean_farming_data['Total Profit (Rs.)'] = clean_farming_data['Total Sales Revenue (Rs.)'] - clean_farming_data['Total Investment (Rs.)']
clean_farming_data['ROI Score (%)'] = (clean_farming_data['Total Profit (Rs.)'] / clean_farming_data['Total Investment (Rs.)']) * 100

print("hi")